In [1]:
from ecnet.model import load_model

cn_model = load_model('_models/cn.pt')
ysi_model = load_model('_models/ysi.pt')
kv_model = load_model('_models/kv.pt')
lhv_model = load_model('_models/lhv.pt')
cp_model = load_model('_models/cp.pt')

with open('_models/cn_desc.txt', 'r') as txt_file:
    desc_cn = txt_file.readlines()
txt_file.close()
desc_cn = [int(i) for i in desc_cn]
with open('_models/ysi_desc.txt', 'r') as txt_file:
    desc_ysi = txt_file.readlines()
txt_file.close()
desc_ysi = [int(i) for i in desc_ysi]
with open('_models/kv_desc.txt', 'r') as txt_file:
    desc_kv = txt_file.readlines()
txt_file.close()
desc_kv = [int(i) for i in desc_kv]
with open('_models/lhv_desc.txt', 'r') as txt_file:
    desc_lhv = txt_file.readlines()
txt_file.close()
desc_lhv = [int(i) for i in desc_lhv]
with open('_models/cp_desc.txt', 'r') as txt_file:
    desc_cp = txt_file.readlines()
txt_file.close()
desc_cp = [int(i) for i in desc_cp]

In [2]:
from csv import DictReader

with open('_data/experimental_blends.csv', 'r') as csv_file:
    reader = DictReader(csv_file)
    rows = [r for r in reader]
csv_file.close()

blend_1 = [r for r in rows if r['Blend ID'] == '0']
blend_1_exp_cn = float(blend_1[0]['CN'])
blend_1_exp_ysi = float(blend_1[0]['YSI'])
blend_1_exp_kv = float(blend_1[0]['KV (cSt)'])
blend_1_exp_lhv = float(blend_1[0]['LHV'])
blend_2 = [r for r in rows if r['Blend ID'] == '1']
blend_2_exp_cn = float(blend_2[0]['CN'])
blend_2_exp_ysi = float(blend_2[0]['YSI'])
blend_2_exp_kv = float(blend_2[0]['KV (cSt)'])
blend_2_exp_lhv = float(blend_2[0]['LHV'])

blend_1_smiles = [r['SMILES'] for r in blend_1]
blend_2_smiles = [r['SMILES'] for r in blend_2]

blend_1_proportions = [float(r['Proportion']) for r in blend_1]
blend_2_proportions = [float(r['Proportion']) for r in blend_2]

In [3]:
from ecnet.datasets import QSPRDataset

blend_1_ds = QSPRDataset(smiles=blend_1_smiles, target_vals=[0 for _ in range(len(blend_1_smiles))],
                         backend='alvadesc')
blend_2_ds = QSPRDataset(smiles=blend_2_smiles, target_vals=[0 for _ in range(len(blend_2_smiles))],
                         backend='alvadesc')

In [4]:
from copy import deepcopy

b1ds_cn = deepcopy(blend_1_ds)
b1ds_cn.set_desc_index(desc_cn)
b1ds_ysi = deepcopy(blend_1_ds)
b1ds_ysi.set_desc_index(desc_ysi)
b1ds_kv = deepcopy(blend_1_ds)
b1ds_kv.set_desc_index(desc_kv)
b1ds_lhv = deepcopy(blend_1_ds)
b1ds_lhv.set_desc_index(desc_lhv)
b1ds_cp = deepcopy(blend_1_ds)
b1ds_cp.set_desc_index(desc_cp)

b2ds_cn = deepcopy(blend_2_ds)
b2ds_cn.set_desc_index(desc_cn)
b2ds_ysi = deepcopy(blend_2_ds)
b2ds_ysi.set_desc_index(desc_ysi)
b2ds_kv = deepcopy(blend_2_ds)
b2ds_kv.set_desc_index(desc_kv)
b2ds_lhv = deepcopy(blend_2_ds)
b2ds_lhv.set_desc_index(desc_lhv)
b2ds_cp = deepcopy(blend_2_ds)
b2ds_cp.set_desc_index(desc_cp)

In [5]:
blend_1_cn_preds = cn_model(b1ds_cn.desc_vals).detach().numpy()
blend_1_ysi_preds = ysi_model(b1ds_ysi.desc_vals).detach().numpy()
blend_1_kv_preds = kv_model(b1ds_kv.desc_vals).detach().numpy()
blend_1_lhv_preds = lhv_model(b1ds_lhv.desc_vals).detach().numpy()
blend_1_cp_preds = cp_model(b1ds_cp.desc_vals).detach().numpy()

blend_2_cn_preds = cn_model(b2ds_cn.desc_vals).detach().numpy()
blend_2_ysi_preds = ysi_model(b2ds_ysi.desc_vals).detach().numpy()
blend_2_kv_preds = kv_model(b2ds_kv.desc_vals).detach().numpy()
blend_2_lhv_preds = lhv_model(b2ds_lhv.desc_vals).detach().numpy()
blend_2_cp_preds = cp_model(b2ds_cp.desc_vals).detach().numpy()

In [6]:
from ecnet.blends.predict import cetane_number, yield_sooting_index, kinematic_viscosity,\
    lower_heating_value, cloud_point

b1_cn = cetane_number([cn[0] for cn in blend_1_cn_preds], blend_1_proportions)
b1_ysi = yield_sooting_index([ysi[0] for ysi in blend_1_ysi_preds], blend_1_proportions)
b1_kv = kinematic_viscosity([kv[0] for kv in blend_1_kv_preds], blend_1_proportions)
b1_lhv = lower_heating_value([lhv[0] for lhv in blend_1_lhv_preds], blend_1_proportions)
b1_cp = cloud_point([cp[0] for cp in blend_1_cp_preds], blend_1_proportions)

b2_cn = cetane_number([cn[0] for cn in blend_2_cn_preds], blend_2_proportions)
b2_ysi = yield_sooting_index([ysi[0] for ysi in blend_2_ysi_preds], blend_2_proportions)
b2_kv = kinematic_viscosity([kv[0] for kv in blend_2_kv_preds], blend_2_proportions)
b2_lhv = lower_heating_value([lhv[0] for lhv in blend_2_lhv_preds], blend_2_proportions)
b2_cp = cloud_point([cp[0] for cp in blend_2_cp_preds], blend_2_proportions)

In [7]:
print(f'Blend 1 CN: {blend_1_exp_cn}, {b1_cn}')
print(f'Blend 1 YSI: {blend_1_exp_ysi}, {b1_ysi}')
print(f'Blend 1 KV: {blend_1_exp_kv}, {b1_kv}')
print(f'Blend 1 LHV: {blend_1_exp_lhv}, {b1_lhv}')
print(f'Blend 1 Pred. CP: {b1_cp}')

print(f'Blend 2 CN: {blend_2_exp_cn}, {b2_cn}')
print(f'Blend 2 YSI: {blend_2_exp_ysi}, {b2_ysi}')
print(f'Blend 2 KV: {blend_2_exp_kv}, {b2_kv}')
print(f'Blend 2 LHV: {blend_2_exp_lhv}, {b2_lhv}')
print(f'Blend 2 Pred. CP: {b2_cp}')

Blend 1 CN: 52.0, 53.316422615051266
Blend 1 YSI: 80.0, 193.96403274536132
Blend 1 KV: 4.054, 4.962754118958515
Blend 1 LHV: 33.0, 37.05621078491211
Blend 1 Pred. CP: 8.778608826456681
Blend 2 CN: 73.0, 74.96247550964355
Blend 2 YSI: 2.1, 16.92345788002014
Blend 2 KV: 1.19, 1.0928216418441432
Blend 2 LHV: 19.3, 17.97402318954468
Blend 2 Pred. CP: 0.0


In [8]:
from ecnet.datasets import load_cn, load_ysi, load_kv, load_cp, load_lhv

_, cn_tar = load_cn()
_, ysi_tar = load_ysi()
_, kv_tar = load_kv()
_, cp_tar = load_cp()
_, lhv_tar = load_lhv()
scale_cn = (min(cn_tar)[0], max(cn_tar)[0])
scale_ysi = (min(ysi_tar)[0], max(ysi_tar)[0])
scale_kv = (min(kv_tar)[0], max(kv_tar)[0])
scale_cp = (min(cp_tar)[0], max(cp_tar)[0])
scale_lhv = (min(lhv_tar)[0], max(lhv_tar)[0])

print('Relative errors:')
print(f'Blend 1 CN: {abs(blend_1_exp_cn - b1_cn) / (scale_cn[1] - scale_cn[0])}')
print(f'Blend 1 YSI: {abs(blend_1_exp_ysi - b1_ysi) / (scale_ysi[1] - scale_ysi[0])}')
print(f'Blend 1 KV: {abs(blend_1_exp_kv - b1_kv) / (scale_kv[1] - scale_kv[0])}')
print(f'Blend 1 LHV: {abs(blend_1_exp_lhv - b1_lhv) / (scale_lhv[1] - scale_lhv[0])}')
print(f'Blend 2 CN: {abs(blend_2_exp_cn - b2_cn) / (scale_cn[1] - scale_cn[0])}')
print(f'Blend 2 YSI: {abs(blend_2_exp_ysi - b2_ysi) / (scale_ysi[1] - scale_ysi[0])}')
print(f'Blend 2 KV: {abs(blend_2_exp_kv - b2_kv) / (scale_kv[1] - scale_kv[0])}')
print(f'Blend 2 LHV: {abs(blend_2_exp_lhv - b2_lhv) / (scale_lhv[1] - scale_lhv[0])}')

Relative errors:
Blend 1 CN: 0.01316422615051266
Blend 1 YSI: 0.08492103781323497
Blend 1 KV: 0.1370339162432166
Blend 1 LHV: 0.10140526962280276
Blend 2 CN: 0.019624755096435534
Blend 2 YSI: 0.011045795737719927
Blend 2 KV: 0.014653832884350192
Blend 2 LHV: 0.03314942026138299


In [9]:
from numpy import mean

relative_errors = [
    abs(blend_1_exp_cn - b1_cn) / (scale_cn[1] - scale_cn[0]),
    abs(blend_1_exp_ysi - b1_ysi) / (scale_ysi[1] - scale_ysi[0]),
    abs(blend_1_exp_kv - b1_kv) / (scale_kv[1] - scale_kv[0]),
    abs(blend_1_exp_lhv - b1_lhv) / (scale_lhv[1] - scale_lhv[0]),
    abs(blend_2_exp_cn - b2_cn) / (scale_cn[1] - scale_cn[0]),
    abs(blend_2_exp_ysi - b2_ysi) / (scale_ysi[1] - scale_ysi[0]),
    abs(blend_2_exp_kv - b2_kv) / (scale_kv[1] - scale_kv[0]),
    abs(blend_2_exp_lhv - b2_lhv) / (scale_lhv[1] - scale_lhv[0])
]
print('Average relative error: {}%'.format(mean(relative_errors) * 100))
print('Average err, POME: {}%'.format(mean(relative_errors[4:]) * 100))
print('Average err, SBD: {}%'.format(mean(relative_errors[:4]) * 100))

Average relative error: 5.187478172620695%
Average err, POME: 1.9618450994972163%
Average err, SBD: 8.413111245744174%
